In [6]:
#Extract games
import asyncio
import chess
import chess.pgn
import chess.engine

async def find_first_blunder(game):
    transport, engine = await chess.engine.popen_uci("../stockfish")
    sf_prev_eval = None
    sf_eval = None
    counter = 0

    while not game.is_end() and \
          counter < 30 and \
          (not(sf_eval) or not sf_eval.is_mate()) and \
          (not(sf_eval and sf_prev_eval) or not abs(sf_eval.relative.score() - sf_prev_eval.relative.score()) >= 200):

        sf_prev_eval = sf_eval
        info = await engine.analyse(game.board(), chess.engine.Limit(time=0.1))
        sf_eval = info["score"]
        game = game.next()
        counter += 1

    return counter

games = []

with open("../data/2020-11.pgn") as f:
    while len(games) < 10:
        games.append(chess.pgn.read_game(f))

games_list = []
for game in games:
    white_name = game.headers['White']
    black_name = game.headers['Black']
    white_elo = game.headers['WhiteElo']
    black_elo = game.headers['BlackElo']
    result = game.headers['Result']
    date = game.headers['UTCDate']
    start_time = game.headers['UTCTime']
    #TODO: endtime calculation based on time format and moves played
    end_time = start_time
    first_blunder = await find_first_blunder(game)
    

    games_list.append([white_name, black_name, white_elo, black_elo, result, date, start_time, end_time, first_blunder])

In [12]:
#Turn into pandas dataframe
import pandas as pd

df = pd.DataFrame(games_list, columns = ["White", "Black", "WhiteElo", "BlackElo", "Result", "UTCDate", "UTCStart", "UTCEnd", "FirstBlunder"])

df

,White,Black,WhiteElo,BlackElo,Result,UTCDate,UTCStart,UTCEnd,FirstBlunder
0,bernec,glebmai,1770,1858,0-1,2020.11.01,00:00:00,00:00:00,6
1,paranakulmichel,LordMorpheus,1964,1798,1-0,2020.11.01,00:00:00,00:00:00,8
2,eli9A,PedroSachica,1519,1864,0-1,2020.11.01,00:00:00,00:00:00,28
3,mini10201,jrdh11,1957,1211,1-0,2020.11.01,00:00:00,00:00:00,13
4,Pevojed,luisulloa,1305,1976,0-1,2020.11.01,00:00:00,00:00:00,22
5,rosaliogomez,Dennis_Vasquez,1985,1331,1-0,2020.11.01,00:00:00,00:00:00,1
6,Ares-Michael,chessnaturaleza,1344,1988,0-1,2020.11.01,00:00:00,00:00:00,0
7,KEERT7,reyrajam_2008,2009,1500,1-0,2020.11.01,00:00:00,00:00:00,1
8,DelaCruz007,matiasgGM,1750,2053,0-1,2020.11.01,00:00:00,00:00:00,5
9,DiableRouge_2,MecanismoAlterado,2042,1725,0-1,2020.11.01,00:00:00,00:00:00,0
